In [2]:
import os
import json
import pandas as pd
from IPython.display import display, HTML

In [3]:
# obsolete
def find_best_config_str(exp_path, mode='retrieval', score_by='MAP'):
    l = []
    # find al files called log.txt, recursively
    for root, dirs, files in os.walk(exp_path):
        for file in files:
            if file.endswith('log.txt'):
                # open file, and for each line test if it contains the string "result for verification run - "
                with open(os.path.join(root, file)) as f:
                    for line in f:
                        if f'result for {mode} run' in line:
                            score = float(line.split(f'{score_by}: ')[1].split(' ')[0])
                            # now, get string between the brackets { and }
                            configs = '{' + line.split('{')[1].split('}')[0] + '}'
                            # replace single quotes with double quotes
                            configs = configs.replace("'", '"')
                            # convert each boolean value to string
                            configs = configs.replace(': True', ': "True"')
                            configs = configs.replace(': False', ': "False"')
                            # convert to json
                            config = json.loads(configs)
                            l.append((score, config))

    # sort the list by the first element, which is the score
    l.sort(key=lambda x: x[0], reverse=True)
    return l

In [105]:


def find_best_config_fp(exp_path, mode='retrieval', m1='MAP', m2='R@5', sort_idx=0):
    configs = []
    # Recursively find all files named log.txt
    for root, _, files in os.walk(exp_path):
        for file in files:
            if file.endswith('log.txt'):
                # Read the file and extract configurations and scores
                with open(os.path.join(root, file)) as f:
                    for line in f:
                        if f'for {mode} run' in line.lower():
                            l = line.split(f': ')
                            v1 = line.split(f'{m1}: ')[1][0:6]
                            v2 = line.split(f'{m2}: ')[1][0:6]
                            score1, score2 = [float(v1), float(v2)]
                            if mode == 'verification':
                                config_path = os.path.join(f.name, '..', 'config.json')
                            elif mode == 'retrieval':
                                config_path = os.path.join(f.name, '..', '..', 'config.json')
                            with open(config_path) as config_file:
                                config = json.load(config_file)
                            configs.append(((m1, score1), (m2, score2), config))
    # Sort configurations by score in descending order
    configs.sort(key=lambda x: x[sort_idx][1], reverse=True)
    return configs



mode = 'retrieval'
m1, m2 = 'MAP', 'R@5'
exp_path = 'experiments-data/split-dev/'+mode
configs_r = find_best_config_fp(exp_path, mode, m1, m2)

mode = 'verification'
m1, m2 ='Macro-F1', 'Strict-Macro-F1'
exp_path = 'experiments-data/split-dev/'+mode
configs_v = find_best_config_fp(exp_path, mode, m1, m2)

print(configs_r)
print(configs_v)

[(('MAP', 0.6877), ('R@5', 0.754), {'blind_run': False, 'split': 'dev', 'preprocess': True, 'add_author_name': True, 'add_author_bio': False, 'retriever_label': 'TERRIER', 'retriever_k': 5, 'out_dir': './experiments-data/split-dev/retrieval/TERRIER/pre-name-nobio', 'fingerprint_r': 'pre-name-nobio'}), (('MAP', 0.6743), ('R@5', 0.7474), {'blind_run': False, 'split': 'dev', 'preprocess': False, 'add_author_name': False, 'add_author_bio': False, 'retriever_label': 'OPENAI', 'retriever_k': 5, 'out_dir': './experiments-data/split-dev/retrieval/OPENAI/nopre-noname-nobio', 'fingerprint_r': 'nopre-noname-nobio'}), (('MAP', 0.6711), ('R@5', 0.7277), {'blind_run': False, 'split': 'dev', 'preprocess': True, 'add_author_name': False, 'add_author_bio': True, 'retriever_label': 'OPENAI', 'retriever_k': 5, 'out_dir': './experiments-data/split-dev/retrieval/OPENAI/pre-noname-bio', 'fingerprint_r': 'pre-noname-bio'}), (('MAP', 0.6594), ('R@5', 0.7519), {'blind_run': False, 'split': 'dev', 'preprocess':

In [4]:
exp_path = 'data-out/experiments/test-dev'

# # print the top 10 results
# mode = 'retrieval'
# score_by='MAP'
# configs = find_best_config(exp_path, mode, score_by)
# for i, (f1, config) in enumerate(configs[:2]):
#     print(f'{i}. {score_by} score: {f1:.4f}, config: {config}')

# print the top 10 results
mode = 'verification'
score_by='Strict-F1'
configs = find_best_config_str(exp_path, mode, score_by)
for i, (score, config) in enumerate(configs[:]):
    print(f'{i}. {score_by} score: {score:.4f}, config: {config}')


0. Strict-F1 score: 0.9230, config: {'blind_run': 'False', 'split': 'dev', 'add_author_name': 'False', 'add_author_bio': 'False', 'retriever_k': 5, 'out_dir': './data-out/experiments/test-dev/retr-OPENAI/nopre-noname-nobio', 'preprocess': 'False', 'retriever_label': 'OPENAI', 'verifier_label': 'OPENAI', 'normalize_scores': 'False', 'scale': 'False', 'ignore_nei': 'True', 'fingerprint': 'nonorm-noscale-ignore_nei'}
1. Strict-F1 score: 0.9230, config: {'blind_run': 'False', 'split': 'dev', 'add_author_name': 'False', 'add_author_bio': 'False', 'retriever_k': 5, 'out_dir': './data-out/experiments/test-dev/retr-OPENAI/nopre-noname-nobio', 'preprocess': 'False', 'retriever_label': 'OPENAI', 'verifier_label': 'OPENAI', 'normalize_scores': 'False', 'scale': 'False', 'ignore_nei': 'False', 'fingerprint': 'nonorm-noscale-noignore_nei'}
2. Strict-F1 score: 0.9230, config: {'blind_run': 'False', 'split': 'dev', 'add_author_name': 'False', 'add_author_bio': 'False', 'retriever_k': 5, 'out_dir': '.

In [5]:
configs = find_best_config_str(exp_path, mode, score_by)
for i, (score, config) in enumerate(configs[:]):
    if config['retriever_label'] == 'TERRIER':
        print(f'{mode} - {i}. {score_by} score: {score:.4f}, config: {config["out_dir"]}/{config["fingerprint"]} - name:{config["add_author_name"]} bio:{config["add_author_bio"]} {config["verifier_label"]}')

verification - 6. Strict-F1 score: 0.8821, config: ./data-out/experiments/test-dev/retr-TERRIER/pre-name-nobio/nonorm-noscale-ignore_nei - name:True bio:False OPENAI
verification - 7. Strict-F1 score: 0.8821, config: ./data-out/experiments/test-dev/retr-TERRIER/pre-name-nobio/nonorm-noscale-noignore_nei - name:True bio:False OPENAI
verification - 8. Strict-F1 score: 0.8821, config: ./data-out/experiments/test-dev/retr-TERRIER/pre-name-nobio/nonorm-scale-ignore_nei - name:True bio:False OPENAI
verification - 9. Strict-F1 score: 0.8821, config: ./data-out/experiments/test-dev/retr-TERRIER/pre-name-nobio/nonorm-scale-noignore_nei - name:True bio:False OPENAI
verification - 10. Strict-F1 score: 0.8821, config: ./data-out/experiments/test-dev/retr-TERRIER/pre-name-nobio/norm-noscale-ignore_nei - name:True bio:False OPENAI
verification - 11. Strict-F1 score: 0.8821, config: ./data-out/experiments/test-dev/retr-TERRIER/pre-name-nobio/norm-noscale-noignore_nei - name:True bio:False OPENAI
veri

In [115]:
mode = 'retrieval'
m1, m2 = 'MAP', 'R@5'
exp_path = 'experiments-data/split-dev/'+mode
configs = find_best_config_fp(exp_path, mode, m1, m2)

# Create a list of dictionaries to store the data
data = []

for i, ((lab1, score1), (lab2, score2), config) in enumerate(configs[:]):
    config_link = f'<a href="{config["out_dir"]}/config.json" target="_blank">Link</a>'
    data.append({
        'Rank': i + 1,
        lab1: score1,
        lab2: score2,
        'Retrieval': "PyTerrier" if config['retriever_label'] == "TERRIER" else "Embeddings", 
        'Preprocess': config['preprocess'],
        'Author Name': config['add_author_bio'],
        'Author Bio': config['add_author_name'],
        'Config': config_link
    })

# Convert the list of dictionaries to a DataFrame
df_retrieval = pd.DataFrame(data)

# Display the DataFrame with clickable links
display(HTML(df_retrieval.to_html(escape=False)))

,Rank,MAP,R@5,Retrieval,Preprocess,Author Name,Author Bio,Config
0,1,0.6877,0.7540,PyTerrier,True,False,True,Link
1,2,0.6743,0.7474,Embeddings,False,False,False,Link
2,3,0.6711,0.7277,Embeddings,True,True,False,Link
3,4,0.6594,0.7519,PyTerrier,True,True,True,Link
4,5,0.6594,0.7519,PyTerrier,True,True,True,Link
5,6,0.6572,0.7081,PyTerrier,True,False,False,Link
6,7,0.6432,0.7168,Embeddings,True,True,True,Link
7,8,0.6428,0.7168,PyTerrier,False,True,False,Link
8,9,0.6415,0.7102,Embeddings,True,False,False,Link
9,10,0.6407,0.7081,PyTerrier,False,False,False,Link


In [124]:
import re
def color_t_f_latex(s):
    # wrap the word "True" in blue, and "False" in red
    s = re.sub(r'(?<!\\)True', r'\\textcolor{blue}{True}', s)
    s = re.sub(r'(?<!\\)False', r'\\textcolor{red}{False}', s)
    return s

In [125]:
latex_str = df_retrieval.drop('Config', axis=1).to_latex(index=False)
print(color_t_f_latex(latex_str))

\begin{tabular}{rrrlrrr}
\toprule
Rank & MAP & R@5 & Retrieval & Preprocess & Author Name & Author Bio \\
\midrule
1 & 0.687700 & 0.754000 & PyTerrier & \textcolor{blue}{True} & \textcolor{red}{False} & \textcolor{blue}{True} \\
2 & 0.674300 & 0.747400 & Embeddings & \textcolor{red}{False} & \textcolor{red}{False} & \textcolor{red}{False} \\
3 & 0.671100 & 0.727700 & Embeddings & \textcolor{blue}{True} & \textcolor{blue}{True} & \textcolor{red}{False} \\
4 & 0.659400 & 0.751900 & PyTerrier & \textcolor{blue}{True} & \textcolor{blue}{True} & \textcolor{blue}{True} \\
5 & 0.659400 & 0.751900 & PyTerrier & \textcolor{blue}{True} & \textcolor{blue}{True} & \textcolor{blue}{True} \\
6 & 0.657200 & 0.708100 & PyTerrier & \textcolor{blue}{True} & \textcolor{red}{False} & \textcolor{red}{False} \\
7 & 0.643200 & 0.716800 & Embeddings & \textcolor{blue}{True} & \textcolor{blue}{True} & \textcolor{blue}{True} \\
8 & 0.642800 & 0.716800 & PyTerrier & \textcolor{red}{False} & \textcolor{blue}{True

In [111]:
# find best config for verification overall
mode = 'verification'
m1, m2 ='Macro-F1', 'Strict-Macro-F1'
exp_path = 'experiments-data/split-dev/'+mode
configs = find_best_config_fp(exp_path, mode, m1, m2)

# Create a list of dictionaries to store the data
data = []

for i, ((lab1, score1), (lab2, score2), config) in enumerate(configs[:]):
    config_link = f'<a href="{config["out_dir"]}/config.json" target="_blank">Link</a>'
    data.append({
        'Rank': i + 1,
        lab1: score1,
        lab2: score2,
        'Retrieval': "PyTerrier" if config['retriever_label'] == "TERRIER" else "Embeddings",
        'Verification': config['verifier_label'],
        'Preprocess': config['preprocess'],
        'Author Name': config['add_author_bio'],
        'Author Bio': config['add_author_name'],
        'Scale': config['scale'],
        'Normalize': config['normalize_scores'],
        'Ignore NEI': config['ignore_nei'],
        'Config': config_link
    })

# Convert the list of dictionaries to a DataFrame
df_verification = pd.DataFrame(data)

# Display the DataFrame with clickable links
display(HTML(df_verification.to_html(escape=False)))

,Rank,Macro-F1,Strict-Macro-F1,Retrieval,Verification,Preprocess,Author Name,Author Bio,Scale,Normalize,Ignore NEI,Config
0,1,0.8716,0.8564,Embeddings,OPENAI,False,True,True,False,False,True,Link
1,2,0.8716,0.8564,Embeddings,OPENAI,False,True,True,False,False,False,Link
2,3,0.8716,0.8564,Embeddings,OPENAI,False,True,True,True,False,True,Link
3,4,0.8716,0.8564,Embeddings,OPENAI,False,True,True,False,True,True,Link
4,5,0.8716,0.8564,Embeddings,OPENAI,False,True,True,False,True,False,Link
5,6,0.8716,0.8564,Embeddings,OPENAI,False,True,True,True,True,True,Link
6,7,0.8716,0.8716,PyTerrier,OPENAI,False,True,True,False,False,True,Link
7,8,0.8716,0.8716,PyTerrier,OPENAI,False,True,True,False,False,False,Link
8,9,0.8716,0.8716,PyTerrier,OPENAI,False,True,True,True,False,True,Link
9,10,0.8716,0.8716,PyTerrier,OPENAI,False,True,True,True,False,False,Link


In [126]:
latex_str = df_verification.drop('Config', axis=1).to_latex(index=False)
print(color_t_f_latex(latex_str))

\begin{tabular}{rrrllrrrrrr}
\toprule
Rank & Macro-F1 & Strict-Macro-F1 & Retrieval & Verification & Preprocess & Author Name & Author Bio & Scale & Normalize & Ignore NEI \\
\midrule
1 & 0.871600 & 0.856400 & Embeddings & OPENAI & \textcolor{red}{False} & \textcolor{blue}{True} & \textcolor{blue}{True} & \textcolor{red}{False} & \textcolor{red}{False} & \textcolor{blue}{True} \\
2 & 0.871600 & 0.856400 & Embeddings & OPENAI & \textcolor{red}{False} & \textcolor{blue}{True} & \textcolor{blue}{True} & \textcolor{red}{False} & \textcolor{red}{False} & \textcolor{red}{False} \\
3 & 0.871600 & 0.856400 & Embeddings & OPENAI & \textcolor{red}{False} & \textcolor{blue}{True} & \textcolor{blue}{True} & \textcolor{blue}{True} & \textcolor{red}{False} & \textcolor{blue}{True} \\
4 & 0.871600 & 0.856400 & Embeddings & OPENAI & \textcolor{red}{False} & \textcolor{blue}{True} & \textcolor{blue}{True} & \textcolor{red}{False} & \textcolor{blue}{True} & \textcolor{blue}{True} \\
5 & 0.871600 & 0.856

In [130]:
# find best config for verification overall
mode = 'verification'
m1, m2 ='Macro-F1', 'Strict-Macro-F1'
exp_path = 'experiments-data/split-dev/'+mode
configs = find_best_config_fp(exp_path, mode, m1, m2)

no_ext_data = []
ext_data = []
for item in configs:
    config = item[2]
    if config['add_author_name'] or config['add_author_bio']:
        ext_data.append(item)
    else:
        no_ext_data.append(item)

no_ext_data = sorted(no_ext_data, key=lambda x: x[0][1], reverse=True)
ext_data = sorted(ext_data, key=lambda x: x[0][1], reverse=True)

In [131]:
# Create a list of dictionaries to store the data
data = []

for i, ((lab1, score1), (lab2, score2), config) in enumerate(no_ext_data[:]):
    config_link = f'<a href="{config["out_dir"]}/config.json" target="_blank">Link</a>'
    data.append({
        'Rank': i + 1,
        lab1: score1,
        lab2: score2,
        'Retrieval': "PyTerrier" if config['retriever_label'] == "TERRIER" else "Embeddings",
        'Verification': config['verifier_label'],
        'Preprocess': config['preprocess'],
        # 'Author Name': config['add_author_bio'],
        # 'Author Bio': config['add_author_name'],
        'Scale': config['scale'],
        'Normalize': config['normalize_scores'],
        'Ignore NEI': config['ignore_nei'],
        'Config': config_link
    })

# Convert the list of dictionaries to a DataFrame
df_verification = pd.DataFrame(data)

# Display the DataFrame with clickable links
display(HTML(df_verification.to_html(escape=False)))

,Rank,Macro-F1,Strict-Macro-F1,Retrieval,Verification,Preprocess,Scale,Normalize,Ignore NEI,Config
0,1,0.8716,0.8564,Embeddings,OPENAI,False,False,False,True,Link
1,2,0.8716,0.8564,Embeddings,OPENAI,False,False,False,False,Link
2,3,0.8716,0.8564,Embeddings,OPENAI,False,True,False,True,Link
3,4,0.8716,0.8564,Embeddings,OPENAI,False,False,True,True,Link
4,5,0.8716,0.8564,Embeddings,OPENAI,False,False,True,False,Link
5,6,0.8716,0.8564,Embeddings,OPENAI,False,True,True,True,Link
6,7,0.8716,0.8716,PyTerrier,OPENAI,False,False,False,True,Link
7,8,0.8716,0.8716,PyTerrier,OPENAI,False,False,False,False,Link
8,9,0.8716,0.8716,PyTerrier,OPENAI,False,True,False,True,Link
9,10,0.8716,0.8716,PyTerrier,OPENAI,False,True,False,False,Link


In [132]:
# Create a list of dictionaries to store the data
data = []

for i, ((lab1, score1), (lab2, score2), config) in enumerate(ext_data[:]):
    config_link = f'<a href="{config["out_dir"]}/config.json" target="_blank">Link</a>'
    data.append({
        'Rank': i + 1,
        lab1: score1,
        lab2: score2,
        'Retrieval': "PyTerrier" if config['retriever_label'] == "TERRIER" else "Embeddings",
        'Verification': config['verifier_label'],
        'Preprocess': config['preprocess'],
        'Author Name': config['add_author_bio'],
        'Author Bio': config['add_author_name'],
        'Scale': config['scale'],
        'Normalize': config['normalize_scores'],
        'Ignore NEI': config['ignore_nei'],
        'Config': config_link
    })

# Convert the list of dictionaries to a DataFrame
df_verification = pd.DataFrame(data)

# Display the DataFrame with clickable links
display(HTML(df_verification.to_html(escape=False)))

,Rank,Macro-F1,Strict-Macro-F1,Retrieval,Verification,Preprocess,Author Name,Author Bio,Scale,Normalize,Ignore NEI,Config
0,1,0.8716,0.8564,Embeddings,OPENAI,False,True,True,False,False,True,Link
1,2,0.8716,0.8564,Embeddings,OPENAI,False,True,True,False,False,False,Link
2,3,0.8716,0.8564,Embeddings,OPENAI,False,True,True,True,False,True,Link
3,4,0.8716,0.8564,Embeddings,OPENAI,False,True,True,False,True,True,Link
4,5,0.8716,0.8564,Embeddings,OPENAI,False,True,True,False,True,False,Link
5,6,0.8716,0.8564,Embeddings,OPENAI,False,True,True,True,True,True,Link
6,7,0.8716,0.8716,PyTerrier,OPENAI,False,True,True,False,False,True,Link
7,8,0.8716,0.8716,PyTerrier,OPENAI,False,True,True,False,False,False,Link
8,9,0.8716,0.8716,PyTerrier,OPENAI,False,True,True,True,False,True,Link
9,10,0.8716,0.8716,PyTerrier,OPENAI,False,True,True,True,False,False,Link


In [ ]:
for i, (f1, config) in enumerate(ext_data):
    print(f'{mode} using ext data - {i+1}. {score_by} score: {f1:.4f}, config: {config}')

In [ ]:
for i, (f1, config) in enumerate(no_ext_data):
    print(f'{mode} no ext data - {i+1}. {score_by} score: {f1:.4f}, config: {config}')